In [1]:
import torch

In [2]:
torch.nn.Linear.reset_parameters = lambda x: None

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
src_path = '/home/jue/v2/CodeLlama-34b/'
src_lora_path = '/home/jue/v2/model_ckpts/lora_delay/global_step200'
tgt_path = '/home/jue/v2/CodeLlama-34b-delay-fine-tuned'

In [5]:
AutoTokenizer.from_pretrained(src_path).save_pretrained(tgt_path)

('/home/jue/v2/CodeLlama-34b-delay-fine-tuned/tokenizer_config.json',
 '/home/jue/v2/CodeLlama-34b-delay-fine-tuned/special_tokens_map.json',
 '/home/jue/v2/CodeLlama-34b-delay-fine-tuned/tokenizer.json')

In [6]:
model = AutoModelForCausalLM.from_pretrained(src_path, torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
for i in range(len(model.model.layers)):
    print(f'layer {i} / {len(model.model.layers)}..')
    layer = model.model.layers[i]
    for j in range(4):
        state_dict = torch.load(
            f'{src_lora_path}/layer_{i+1:02d}-model_{j:02d}-model_states.pt',
            map_location='cuda:0'
        )
        q_lora = (state_dict['self_attn.q_proj.lora_B'] @ state_dict['self_attn.q_proj.lora_A'])
        layer.self_attn.q_proj.weight.data[j*q_lora.size(0):(j+1)*q_lora.size(0)] += q_lora.cpu() # * (alpha / r)

        v_lora = (state_dict['self_attn.v_proj.lora_B'] @ state_dict['self_attn.v_proj.lora_A'])
        layer.self_attn.v_proj.weight.data[j*v_lora.size(0):(j+1)*v_lora.size(0)] += v_lora.cpu() # * (alpha / r)

layer 0 / 48..
layer 1 / 48..
layer 2 / 48..
layer 3 / 48..
layer 4 / 48..
layer 5 / 48..
layer 6 / 48..
layer 7 / 48..
layer 8 / 48..
layer 9 / 48..
layer 10 / 48..
layer 11 / 48..
layer 12 / 48..
layer 13 / 48..
layer 14 / 48..
layer 15 / 48..
layer 16 / 48..
layer 17 / 48..
layer 18 / 48..
layer 19 / 48..
layer 20 / 48..
layer 21 / 48..
layer 22 / 48..
layer 23 / 48..
layer 24 / 48..
layer 25 / 48..
layer 26 / 48..
layer 27 / 48..
layer 28 / 48..
layer 29 / 48..
layer 30 / 48..
layer 31 / 48..
layer 32 / 48..
layer 33 / 48..
layer 34 / 48..
layer 35 / 48..
layer 36 / 48..
layer 37 / 48..
layer 38 / 48..
layer 39 / 48..
layer 40 / 48..
layer 41 / 48..
layer 42 / 48..
layer 43 / 48..
layer 44 / 48..
layer 45 / 48..
layer 46 / 48..
layer 47 / 48..


In [8]:
model.save_pretrained(tgt_path, safe_serialization=True)

In [9]:
exit()